<a href="https://colab.research.google.com/github/hrishabh-dev/pythonbasics/blob/main/niftybacktesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# make a ml model to predict nifty 50 high and low price of daily chart

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# scaling
from sklearn.preprocessing import StandardScaler

In [3]:
df=pd.read_csv("/content/Nifty 50 Historical Data.csv")

In [5]:
from IPython import get_ipython
from IPython.display import display
# %%
# make a ml model to predict nifty 50 high and low price of daily chart
# %%
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# scaling
from sklearn.preprocessing import StandardScaler

# Import a wider range of regressors
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor

In [7]:
df

,Date,Price,Open,High,Low,Vol.,Change %
0,25-05-2025,"24,750.70","24,919.35","25,078.95","24,680.60",2.71B,-0.41%
1,18-05-2025,"24,853.15","25,005.35","25,062.95","24,462.40",1.68B,-0.67%
2,11-05-2025,"25,019.80","24,420.10","25,116.25","24,378.85",2.07B,4.21%
3,04-05-2025,"24,008.00","24,419.50","24,526.40","23,935.75",1.67B,-1.39%
4,27-04-2025,"24,346.70","24,070.25","24,589.15","24,054.05",1.52B,1.28%
...,...,...,...,...,...,...,...
694,05-02-2012,"5,381.60","5,379.45","5,427.75","5,322.95",1.11B,1.05%
695,29-01-2012,"5,325.85","5,163.55","5,334.85","5,076.70",1.19B,2.33%
696,22-01-2012,"5,204.70","5,025.35","5,217.00","5,021.35",795.28M,3.09%
697,15-01-2012,"5,048.60","4,844.00","5,064.15","4,827.05",1.02B,3.75%


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      699 non-null    object
 1   Price     699 non-null    object
 2   Open      699 non-null    object
 3   High      699 non-null    object
 4   Low       699 non-null    object
 5   Vol.      699 non-null    object
 6   Change %  699 non-null    object
dtypes: object(7)
memory usage: 38.4+ KB


In [9]:
columns_to_convert = ['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']

for col in columns_to_convert:
    # Remove commas from the string and then convert to float
    if col != 'Change %' and col != 'Vol.': # Apply to columns with commas
        df[col] = df[col].str.replace(',', '', regex=False).astype(float)
    elif col == 'Change %': # Handle percentage sign and potentially commas
        df[col] = df[col].str.replace('%', '', regex=False)
        # Check for commas and remove them if present before converting
        df[col] = df[col].str.replace(',', '', regex=False).astype(float)
    elif col == 'Vol.': # Handle volume column which might have 'M' or 'K'
        # Convert 'M' to millions and 'K' to thousands
        df[col] = df[col].str.replace('M', 'e6', regex=False)
        df[col] = df[col].str.replace('K', 'e3', regex=False)
        # Handle cases where there's no 'M' or 'K' and just a number (potentially with commas)
        df[col] = df[col].str.replace(',', '', regex=False)
        # Convert to numeric, coercing errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')


# Verify the data types after conversion
print("\nData types after conversion:")
df.info()


Data types after conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      699 non-null    object 
 1   Price     699 non-null    float64
 2   Open      699 non-null    float64
 3   High      699 non-null    float64
 4   Low       699 non-null    float64
 5   Vol.      278 non-null    float64
 6   Change %  699 non-null    float64
dtypes: float64(6), object(1)
memory usage: 38.4+ KB


In [11]:
df.drop('Vol.',axis=1,inplace=True)

In [12]:
df.drop('Change %',axis=1,inplace=True)

In [25]:
import pandas as pd

# User inputs
initial_capital = float(input("Enter initial capital: "))
target_pct = float(input("Enter target profit percentage (e.g., 5 for 5%): "))
sl_pct = float(input("Enter stop-loss percentage (e.g., 2 for 2%): "))

# The DataFrame 'df' is already loaded and processed from previous cells.
# Ensure the 'Date' column is in datetime format if it wasn't parsed before.
# Based on the info() output from previous cells, 'Date' is likely an object.
# Let's ensure it's a datetime object.
# Add a check to see if 'Date' column exists before attempting conversion
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
else:
    print("Error: 'Date' column not found in the DataFrame.")
    # You might want to exit or handle this error appropriately
    # For now, we'll continue, but the sort will likely fail or be incorrect
    pass


# Sort data by date
df = df.sort_values('Date').reset_index(drop=True)

# Initialize variables
capital = initial_capital
profitable_days = 0
loss_days = 0
total_trades = 0

# To store trade results
trade_results = []

# Loop through data, skipping the last day since we need next day's open/price (formerly close)
for i in range(len(df)-1):
    # entry_price is at next day's open based on strategy
    next_day_open = df.loc[i+1, 'Open'] if 'Open' in df.columns else None
    # Use 'Price' column for the next day's closing price
    next_day_price = df.loc[i+1, 'Price'] if 'Price' in df.columns else None

    # Check if required columns exist
    if next_day_open is None or next_day_price is None:
        print(f"Skipping day {df.loc[i+1, 'Date']} due to missing 'Open' or 'Price' data.")
        continue # Skip this iteration if data is missing

    # Enter trade at next day's open
    entry_price = next_day_open
    # Define target and stop-loss prices
    target_price = entry_price * (1 + target_pct/100)
    sl_price = entry_price * (1 - sl_pct/100)

    # Next day's price movement (simulate intra-day movement)
    high_price = df.loc[i+1, 'High'] if 'High' in df.columns else None
    low_price = df.loc[i+1, 'Low'] if 'Low' in df.columns else None

    # For simplicity, assume the high and low prices are available for the day
    # If not, we can approximate with open and price prices (less accurate)
    if high_price is None or low_price is None:
        # Fallback: assume the day's high and low are close to open and price
        high_price = max(entry_price, next_day_price)
        low_price = min(entry_price, next_day_price)

    # Simulate intra-day price movement:
    # Check if target or stop-loss is hit during the day
    target_hit = False
    sl_hit = False

    # Determine if target or stop-loss is hit, considering the order
    # This is a simplification. A more accurate simulation would require tick data.
    # Assuming target is checked first if both are possible based on High and Low.

    if high_price >= target_price and low_price <= sl_price:
        # Both could have been hit. Which one happened first?
        # Simplification: Assume target is hit if high reaches target before low reaches sl from open
        # This is not strictly accurate but common in simple backtests.
        # A more rigorous approach would compare ratios (target/open vs open/sl) relative to high/open vs open/low
        # For this code's logic, let's keep it simple based on the existing check order.

        # Check if target is hit
        if high_price >= target_price:
            target_hit = True
        # Check if stop loss is hit
        if low_price <= sl_price:
            sl_hit = True

        # Decide exit based on which was hit, assuming target takes precedence if both are possible
        if target_hit:
             exit_price = target_price
             result = 'Profit'
             profit_percent = target_pct
        elif sl_hit:
             exit_price = sl_price
             result = 'Loss'
             profit_percent = -sl_pct
        else: # Should not happen if both are possible but neither hit
            # Exit at next day's closing price
            exit_price = next_day_price
            profit_percent = ((exit_price - entry_price) / entry_price) * 100
            result = 'Profit' if profit_percent >= 0 else 'Loss' # Use >= for 0% profit/loss

    elif high_price >= target_price:
        # Only target hit
        exit_price = target_price
        result = 'Profit'
        profit_percent = target_pct
    elif low_price <= sl_price:
        # Only stop-loss hit
        exit_price = sl_price
        result = 'Loss'
        profit_percent = -sl_pct
    else:
        # Neither hit, exit at next day's closing price
        exit_price = next_day_price
        # Calculate profit/loss
        profit_percent = ((exit_price - entry_price) / entry_price) * 100
        result = 'Profit' if profit_percent >= 0 else 'Loss' # Use >= for 0% profit/loss


    # Calculate profit or loss in amount
    trade_amount = capital  # Using entire capital for each trade
    profit_loss = trade_amount * (profit_percent / 100)
    capital += profit_loss  # Update capital

    # Record trade outcome
    total_trades += 1
    if profit_percent > 0:
        profitable_days += 1
    else:
        loss_days += 1

    trade_results.append({
        'Date': df.loc[i+1, 'Date'],
        'Entry Price': entry_price,
        'Exit Price': exit_price,
        'Result': result,
        'Profit%': profit_percent,
        'Capital': capital
    })

# Final report
print("\n--- Backtest Summary ---")
print(f"Initial Capital: {initial_capital}")
print(f"Final Capital: {capital:.2f}")
print(f"Total Trades: {total_trades}")
print(f"Profitable Days: {profitable_days}")
print(f"Loss Days: {loss_days}")

Enter initial capital: 20000
Enter target profit percentage (e.g., 5 for 5%): 20
Enter stop-loss percentage (e.g., 2 for 2%): 10

--- Backtest Summary ---
Initial Capital: 20000.0
Final Capital: 44256.62
Total Trades: 698
Profitable Days: 378
Loss Days: 320
Detailed trade results saved to 'trade_results.csv'


In [27]:
import pandas as pd
import numpy as np


# Assuming df is already loaded and cleaned from previous cells

# Ensure the 'Date' column is in datetime format and data is sorted
# Add a check to see if 'Date' column exists before attempting conversion
if 'Date' in df.columns:
    # Use errors='coerce' to turn unparseable dates into NaT (Not a Time)
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
    # Drop rows where Date could not be parsed
    df.dropna(subset=['Date'], inplace=True)
else:
    print("Error: 'Date' column not found in the DataFrame. Cannot proceed.")
    # Exit or handle error appropriately if 'Date' is critical
    exit() # Example: exit the script if Date is missing

# Sort data by date
df = df.sort_values('Date').reset_index(drop=True)

# User inputs
initial_capital = float(input("Enter initial capital: "))
target_pct = float(input("Enter target profit percentage (e.g., 5 for 5%): "))
sl_pct = float(input("Enter stop-loss percentage (e.g., 2 for 2%): "))

# Initialize variables
capital = initial_capital
profitable_days = 0
loss_days = 0
total_trades = 0

# To store trade results
trade_results = []

# Loop through data, skipping the last day since we need next day's open/price
for i in range(len(df)-1):
    # entry_price is at next day's open based on strategy
    next_day_open = df.loc[i+1, 'Open'] if 'Open' in df.columns else None
    # Use 'Price' column for the next day's closing price
    next_day_price = df.loc[i+1, 'Price'] if 'Price' in df.columns else None

    # Check if required columns exist and are not NaN
    if next_day_open is None or pd.isna(next_day_open) or \
       next_day_price is None or pd.isna(next_day_price):
        print(f"Skipping day {df.loc[i+1, 'Date']} due to missing 'Open' or 'Price' data.")
        continue # Skip this iteration if data is missing

    # Enter trade at next day's open
    entry_date = df.loc[i+1, 'Date']
    entry_price = next_day_open

    # Define target and stop-loss prices
    target_price = entry_price * (1 + target_pct/100)
    sl_price = entry_price * (1 - sl_pct/100)

    # Next day's price movement (simulate intra-day movement)
    high_price = df.loc[i+1, 'High'] if 'High' in df.columns else None
    low_price = df.loc[i+1, 'Low'] if 'Low' in df.columns else None

    # For simplicity, assume the high and low prices are available for the day
    # If not, we can approximate with open and price prices (less accurate)
    if high_price is None or pd.isna(high_price) or \
       low_price is None or pd.isna(low_price):
        # Fallback: assume the day's high and low are close to open and price
        high_price = max(entry_price, next_day_price)
        low_price = min(entry_price, next_day_price)


    # Simulate intra-day price movement:
    target_hit = False
    sl_hit = False
    exit_price = None
    result = 'Flat' # Default result if none hit

    # Determine if target or stop-loss is hit, considering the order
    # Assuming target is checked first if both are possible based on High and Low.
    if high_price >= target_price and low_price <= sl_price:
        # Both could have been hit. Which one happened first?
        # Simplification: Assume target hit if high reaches target before low reaches sl from open
        if entry_price <= target_price: # Check if price moved up towards target first
             exit_price = target_price
             result = 'Profit'
             profit_percent = target_pct
        elif entry_price >= sl_price: # Check if price moved down towards stop-loss first
             exit_price = sl_price
             result = 'Loss'
             profit_percent = -sl_pct
        else: # Should not happen if both levels are within range from entry
            # Default to closing price if order is ambiguous
            exit_price = next_day_price
            profit_percent = ((exit_price - entry_price) / entry_price) * 100
            result = 'Profit' if profit_percent >= 0 else 'Loss'


    elif high_price >= target_price:
        # Only target hit
        exit_price = target_price
        result = 'Profit'
        profit_percent = target_pct
    elif low_price <= sl_price:
        # Only stop-loss hit
        exit_price = sl_price
        result = 'Loss'
        profit_percent = -sl_pct
    else:
        # Neither hit, exit at next day's closing price
        exit_price = next_day_price
        # Calculate profit/loss
        profit_percent = ((exit_price - entry_price) / entry_price) * 100
        result = 'Profit' if profit_percent >= 0 else 'Loss' # Use >= for 0% profit/loss

    # Calculate profit or loss in amount
    trade_amount = capital  # Using entire capital for each trade
    profit_loss = trade_amount * (profit_percent / 100)
    capital += profit_loss  # Update capital

    # Record trade outcome
    total_trades += 1
    if profit_percent > 0:
        profitable_days += 1
    else:
        loss_days += 1

    # Store detailed trade result
    trade_results.append({
        'Entry Date': entry_date.strftime('%Y-%m-%d'), # Format date
        'Entry Price': round(entry_price, 2),
        'Exit Date': entry_date.strftime('%Y-%m-%d'), # Exit is on the same day as entry based on this logic
        'Exit Price': round(exit_price, 2),
        'Result': result,
        'Profit%': round(profit_percent, 2),
        'Capital After Trade': round(capital, 2)
    })


# Print detailed trade results
print("\n--- Detailed Trade Results ---")
if trade_results:
    for trade in trade_results:
        print(f"  Date: {trade['Entry Date']}, Entry: {trade['Entry Price']}, Exit: {trade['Exit Price']}, Result: {trade['Result']} ({trade['Profit%']}%), Capital: {trade['Capital After Trade']}")
else:
    print("  No trades were executed based on the defined logic.")


# Final summary report
print("\n--- Backtest Summary ---")
print(f"Initial Capital: {initial_capital}")
print(f"Final Capital: {capital:.2f}")
print(f"Total Trades: {total_trades}")
print(f"Profitable Days: {profitable_days}")
print(f"Loss Days: {loss_days}")

# Optional: Save detailed trade results to CSV
df_results = pd.DataFrame(trade_results)
df_results.to_csv('trade_results_without_indicators.csv', index=False)
print("Detailed trade results saved to 'trade_results_without_indicators.csv'")

Enter initial capital: 50000
Enter target profit percentage (e.g., 5 for 5%): 30
Enter stop-loss percentage (e.g., 2 for 2%): 4

--- Detailed Trade Results ---
  Date: 2012-01-15, Entry: 4844.0, Exit: 5048.6, Result: Profit (4.22%), Capital: 52111.89
  Date: 2012-01-22, Entry: 5025.35, Exit: 5204.7, Result: Profit (3.57%), Capital: 53971.72
  Date: 2012-01-29, Entry: 5163.55, Exit: 5325.85, Result: Profit (3.14%), Capital: 55668.15
  Date: 2012-02-05, Entry: 5379.45, Exit: 5381.6, Result: Profit (0.04%), Capital: 55690.4
  Date: 2012-02-12, Entry: 5382.1, Exit: 5564.3, Result: Profit (3.39%), Capital: 57575.68
  Date: 2012-02-19, Entry: 5561.9, Exit: 5429.3, Result: Loss (-2.38%), Capital: 56203.03
  Date: 2012-02-26, Entry: 5448.1, Exit: 5359.35, Result: Loss (-1.63%), Capital: 55287.48
  Date: 2012-03-04, Entry: 5342.55, Exit: 5333.55, Result: Loss (-0.17%), Capital: 55194.34
  Date: 2012-03-11, Entry: 5420.1, Exit: 5317.9, Result: Loss (-1.89%), Capital: 54153.61
  Date: 2012-03-18,